In [4]:
import re
import pandas as pd
import mysql.connector
import numpy as np
from datetime import datetime

from utils_text import strip_accents

In [5]:
def clean_text(string: str) -> str:
    string = string.lower()
    string = strip_accents(string)
    string = string.strip()
    
    return string 

def to_datetime(string: str) -> str:
    return datetime.strptime(string, '%d/%m/%Y')

# Dataset

## df_viagem

In [17]:
df_viagem = pd.read_csv('./dataset/2020_Viagem.csv', delimiter=";", encoding='latin1')

In [18]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323503 entries, 0 to 323502
Data columns (total 21 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  323503 non-null  int64 
 1   Número da Proposta (PCDP)            323503 non-null  object
 2   Situação                             323503 non-null  object
 3   Viagem Urgente                       323503 non-null  object
 4   Justificativa Urgência Viagem        323442 non-null  object
 5   Código do órgão superior             323503 non-null  int64 
 6   Nome do órgão superior               323503 non-null  object
 7   Código órgão solicitante             323503 non-null  int64 
 8   Nome órgão solicitante               323503 non-null  object
 9   CPF viajante                         323475 non-null  object
 10  Nome                                 323503 non-null  object
 11  Cargo                     

In [19]:
df_viagem = df_viagem.drop_duplicates(subset=['Identificador do processo de viagem'])

In [20]:
df_viagem = df_viagem.drop('Motivo', axis=1)
df_viagem = df_viagem.drop('Valor outros gastos', axis=1)
df_viagem = df_viagem.drop('Valor diárias', axis=1)
df_viagem = df_viagem.drop('Destinos', axis=1)
df_viagem = df_viagem.drop('Justificativa Urgência Viagem', axis=1)
df_viagem = df_viagem.drop('Código órgão solicitante', axis=1)
df_viagem = df_viagem.drop('Nome órgão solicitante', axis=1)
df_viagem = df_viagem.drop('Número da Proposta (PCDP)', axis=1)
df_viagem = df_viagem.reset_index(drop=True)

In [21]:
df_viagem = df_viagem.rename(columns={'Identificador do processo de viagem': 'id_viagem'})

In [22]:
df_viagem = df_viagem.replace({np.nan: "Sem informação"})

In [23]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df_viagem.columns:
    if df_viagem[column].dtypes == 'object':
        df_viagem[column] = df_viagem[column].apply(clean_text)

In [24]:
df_viagem['Período - Data de início'] = df_viagem['Período - Data de início'].apply(to_datetime)
df_viagem['Período - Data de fim'] = df_viagem['Período - Data de fim'].apply(to_datetime)

In [25]:
df_viagem = df_viagem.replace({"nao informado": "sem informacao"})
df_viagem = df_viagem.replace({"-1": "sem informacao"})
df_viagem = df_viagem.replace({-1: "sem informacao"})
df_viagem['Código do órgão superior'] = df_viagem['Código do órgão superior'].replace({"sem informacao": -1})

In [27]:
df_viagem['Valor passagens'] = df_viagem['Valor passagens'].apply(lambda x: float(x.replace(',','.')))

In [28]:
df_remove = df_viagem[df_viagem['CPF viajante'].str.contains(r'ID.*|\*\*\*.000.000-\*\*')]
lista_id_remove = list(df_remove['id_viagem'])
df_viagem = df_viagem.drop(index=df_remove.index)
df_viagem = df_viagem.reset_index(drop=True)

In [29]:
df_viagem.index += 1

In [30]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322007 entries, 1 to 322007
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id_viagem                 322007 non-null  object        
 1   Situação                  322007 non-null  object        
 2   Viagem Urgente            322007 non-null  object        
 3   Código do órgão superior  322007 non-null  int64         
 4   Nome do órgão superior    322007 non-null  object        
 5   CPF viajante              322007 non-null  object        
 6   Nome                      322007 non-null  object        
 7   Cargo                     322007 non-null  object        
 8   Função                    322007 non-null  object        
 9   Descrição Função          322007 non-null  object        
 10  Período - Data de início  322007 non-null  datetime64[ns]
 11  Período - Data de fim     322007 non-null  datetime64[ns]
 12  Va

In [31]:
df_viagem.head()

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Valor passagens
1,16332319,nao realizada,nao,26000,ministerio da educacao,***.211.458-**,claudia correa de andrade oliveira,professor do magisterio superior,sem informacao,sem informacao,2020-01-05,2021-01-04,0.00
2,16386658,realizada,nao,26000,ministerio da educacao,***.708.864-**,alexsandro galeno araujo dantas,professor do magisterio superior,fuc-0001,fuc-0001,2020-03-10,2020-03-12,1053.89
3,16401038,realizada,nao,26000,ministerio da educacao,***.634.280-**,fernando carlos giacomelli,professor do magisterio superior,sem informacao,sem informacao,2020-01-01,2020-09-15,0.00
4,16413295,realizada,nao,26000,ministerio da educacao,***.464.507-**,rodrigo ramos hospodar felippe valverde,sem informacao,sem informacao,sem informacao,2020-03-02,2020-03-02,628.56
5,16415470,realizada,sim,26000,ministerio da educacao,***.126.127-**,martius vicente rodriguez y rodriguez,professor do magisterio superior,cd-0003,cargo comissionado de direcao,2020-01-11,2020-01-23,6903.68


## df_passagem

In [32]:
df_passagem = pd.read_csv('./dataset/2020_Passagem.csv', delimiter=";", encoding='latin1')

In [33]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108981 entries, 0 to 108980
Data columns (total 19 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  108981 non-null  int64 
 1   Número da Proposta (PCDP)            108981 non-null  object
 2   Meio de transporte                   108981 non-null  object
 3   País - Origem ida                    108981 non-null  object
 4   UF - Origem ida                      106597 non-null  object
 5   Cidade - Origem ida                  108981 non-null  object
 6   País - Destino ida                   108981 non-null  object
 7   UF - Destino ida                     106058 non-null  object
 8   Cidade - Destino ida                 108981 non-null  object
 9   País - Origem volta                  108981 non-null  object
 10  UF - Origem volta                    108149 non-null  object
 11  Cidade - Origem volta     

In [34]:
df_passagem = df_passagem[df_passagem['Identificador do processo de viagem'].isin(df_viagem['id_viagem'])]

In [35]:
df_passagem['Valor da passagem'] = df_passagem['Valor da passagem'].apply(lambda x: float(x.replace(',','.')))
df_passagem['Taxa de serviço'] = df_passagem['Taxa de serviço'].apply(lambda x: float(x.replace(',','.')))
df_passagem['Total'] = df_passagem['Valor da passagem'] + df_passagem['Taxa de serviço']

In [36]:
df_passagem = df_passagem.rename(columns={'Identificador do processo de viagem': 'id_viagem'})

In [37]:
df_passagem = df_passagem[~df_passagem['id_viagem'].isin(lista_id_remove)]

In [38]:
df_passagem = df_passagem.drop('Hora da emissão/compra', axis=1)
df_passagem = df_passagem.drop('Número da Proposta (PCDP)', axis=1)
df_passagem = df_passagem.drop('Data da emissão/compra', axis=1)
df_passagem = df_passagem.reset_index(drop=True)

In [39]:
df_passagem.index += 1
df_passagem['id_passagem'] = df_passagem.index

In [40]:
df_passagem = df_passagem.replace({np.nan: "Sem informação"})

In [41]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df_passagem.columns:
    if df_passagem[column].dtypes == 'object':
        df_passagem[column] = df_passagem[column].apply(clean_text)

In [42]:
df_passagem = df_passagem.replace({"nao informado": "sem informacao"})

In [43]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107015 entries, 1 to 107015
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_viagem               107015 non-null  int64  
 1   Meio de transporte      107015 non-null  object 
 2   País - Origem ida       107015 non-null  object 
 3   UF - Origem ida         107015 non-null  object 
 4   Cidade - Origem ida     107015 non-null  object 
 5   País - Destino ida      107015 non-null  object 
 6   UF - Destino ida        107015 non-null  object 
 7   Cidade - Destino ida    107015 non-null  object 
 8   País - Origem volta     107015 non-null  object 
 9   UF - Origem volta       107015 non-null  object 
 10  Cidade - Origem volta   107015 non-null  object 
 11  Pais - Destino volta    107015 non-null  object 
 12  UF - Destino volta      107015 non-null  object 
 13  Cidade - Destino volta  107015 non-null  object 
 14  Valor da passagem   

In [44]:
df_passagem.head()

,id_viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Total,id_passagem
1,16386658,aereo,brasil,rio grande do norte,natal,brasil,distrito federal,brasilia,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,522.04,1.14,523.18,1
2,16386658,aereo,brasil,distrito federal,brasilia,brasil,rio grande do norte,natal,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,529.57,1.14,530.71,2
3,16413295,aereo,brasil,sao paulo,sao paulo,brasil,distrito federal,brasilia,brasil,distrito federal,brasilia,brasil,sao paulo,sao paulo,627.42,1.14,628.56,3
4,16415470,aereo,brasil,rio de janeiro,rio de janeiro,portugal,sem informacao,lisboa,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,4
5,16415470,aereo,portugal,sem informacao,lisboa,brasil,rio de janeiro,rio de janeiro,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,5


# Analise e tabelas secundárias

## Cidade, UF, Pais

In [45]:
df_puc = df_passagem[['País - Origem ida', 'UF - Origem ida',
       'Cidade - Origem ida', 'País - Destino ida', 'UF - Destino ida',
       'Cidade - Destino ida', 'País - Origem volta', 'UF - Origem volta',
       'Cidade - Origem volta', 'Pais - Destino volta', 'UF - Destino volta',
       'Cidade - Destino volta']]

In [46]:
paises = list(set(list(df_puc['País - Origem ida'].unique()) + list(df_puc['País - Destino ida'].unique()) + list(df_puc['País - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [47]:
df_paises= pd.DataFrame(paises, columns=['nome'])
df_paises.index += 1
df_paises['id_paises'] = df_paises.index
df_paises

,nome,id_paises
1,bulgaria,1
2,irlanda,2
3,costa rica,3
4,sri lanka,4
5,sem informacao,5
...,...,...
126,espanha,126
127,russia,127
128,canada,128
129,chipre,129


In [48]:
uf = list(set(list(df_puc['UF - Origem ida'].unique()) + list(df_puc['UF - Destino ida'].unique()) + list(df_puc['UF - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [49]:
df_uf = pd.DataFrame(uf, columns=['nome'])
df_uf.index += 1
df_uf['id_uf'] = df_uf.index
df_uf

,nome,id_uf
1,bulgaria,1
2,paraiba,2
3,irlanda,3
4,pernambuco,4
5,sem informacao,5
...,...,...
87,portugal,87
88,mocambique,88
89,espanha,89
90,russia,90


In [50]:
cidades = list(set(list(df_puc['Cidade - Origem ida'].unique()) + list(df_puc['Cidade - Destino ida'].unique()) + list(df_puc['Cidade - Origem volta'].unique()) + list(df_puc['Pais - Destino volta'].unique())))

In [51]:
df_cidades = pd.DataFrame(cidades, columns=['nome'])
df_cidades.index += 1
df_cidades['id_cidade'] = df_cidades.index
df_cidades

,nome,id_cidade
1,hortolandia,1
2,tbilisi,2
3,xangai,3
4,aalborg,4
5,sofia,5
...,...,...
736,mendoza,736
737,chipre,737
738,russia,738
739,larnaca,739


## Ministério

In [52]:
df_ministerios = df_viagem.copy()
df_ministerios = df_ministerios.filter(['Código do órgão superior', 'Nome do órgão superior'])

In [53]:
df_ministerios = df_ministerios.rename(columns=
                                       {'Código do órgão superior': 'id_ministerio', 
                                        'Nome do órgão superior': 'nome_ministerio'})

In [54]:
df_ministerios = df_ministerios.drop_duplicates(subset=['id_ministerio'])

In [55]:
df_ministerios.head()

,id_ministerio,nome_ministerio
1,26000,ministerio da educacao
6,52000,ministerio da defesa
8,36000,ministerio da saude
30,35000,ministerio das relacoes exteriores
51,30000,ministerio da justica e seguranca publica


## Pessoa

In [56]:
df_pessoa = df_viagem.copy()

In [57]:
df_pessoa = df_pessoa.filter(['CPF viajante', 'Nome', 'Cargo', 'Função', 'Descrição Função'])

In [58]:
df_pessoa.head()

,CPF viajante,Nome,Cargo,Função,Descrição Função
1,***.211.458-**,claudia correa de andrade oliveira,professor do magisterio superior,sem informacao,sem informacao
2,***.708.864-**,alexsandro galeno araujo dantas,professor do magisterio superior,fuc-0001,fuc-0001
3,***.634.280-**,fernando carlos giacomelli,professor do magisterio superior,sem informacao,sem informacao
4,***.464.507-**,rodrigo ramos hospodar felippe valverde,sem informacao,sem informacao,sem informacao
5,***.126.127-**,martius vicente rodriguez y rodriguez,professor do magisterio superior,cd-0003,cargo comissionado de direcao


In [59]:
df_pessoa.drop_duplicates(subset=['CPF viajante'], inplace=True)

In [60]:
df_pessoa = df_pessoa.reset_index(drop=True)

In [61]:
df_pessoa_final = df_pessoa[['Nome', 'CPF viajante']]

In [62]:
df_pessoa_final.index += 1
df_pessoa_final['id_pessoa'] = df_pessoa_final.index

<ipython-input-62-26d9a24bddf6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pessoa_final['id_pessoa'] = df_pessoa_final.index


In [63]:
df_pessoa_final

,Nome,CPF viajante,id_pessoa
1,claudia correa de andrade oliveira,***.211.458-**,1
2,alexsandro galeno araujo dantas,***.708.864-**,2
3,fernando carlos giacomelli,***.634.280-**,3
4,rodrigo ramos hospodar felippe valverde,***.464.507-**,4
5,martius vicente rodriguez y rodriguez,***.126.127-**,5
...,...,...,...
105758,rodrigo campelo crispim,***.922.261-**,105758
105759,vinicius pacheco barcelos,***.679.700-**,105759
105760,cecilia silva gontijo,***.906.656-**,105760
105761,robson linko nunes da silva,***.873.083-**,105761


## Cargo

In [64]:
df_cargo = df_viagem[['Cargo']]

In [65]:
df_cargo = df_cargo.drop_duplicates(subset=['Cargo'])

In [66]:
df_cargo = df_cargo.reset_index(drop=True)

In [67]:
df_cargo.index += 1
df_cargo['id_cargo'] = df_cargo.index

In [68]:
df_cargo

,Cargo,id_cargo
1,professor do magisterio superior,1
2,sem informacao,2
3,pesquisador em saude publica,3
4,professor ens basico tecn tecnologico,4
5,professor tit-livre magist superior,5
...,...,...
686,diretor servidor do banco central,686
687,tecnico do banco central,687
688,presidente do banco central,688
689,diretor do banco central,689


## Função

In [69]:
df_funcao = df_viagem[['Função', 'Descrição Função']]

In [70]:
df_funcao = df_funcao.drop_duplicates(subset=['Função'])

In [71]:
df_funcao = df_funcao.reset_index(drop=True)

In [72]:
df_funcao.index += 1
df_funcao['id_funcao'] = df_funcao.index

In [73]:
df_funcao

,Função,Descrição Função,id_funcao
1,sem informacao,sem informacao,1
2,fuc-0001,fuc-0001,2
3,cd-0003,cargo comissionado de direcao,3
4,ofsuperior,ofsuperior,4
5,of.general,oficiais generais,5
...,...,...,...
138,gts-003,gratificacao temporaria - sipam,138
139,das-1033,sem informacao,139
140,das-1032,sem informacao,140
141,nes-0032,cargos de natureza especial,141


## Meio de transporte

In [74]:
meio = df_passagem['Meio de transporte']

In [75]:
meio = list(meio.unique())

In [76]:
df_meio = pd.DataFrame(meio, columns=['nome'])
df_meio.index += 1

df_meio['id_meio'] = df_meio.index

In [77]:
df_meio

,nome,id_meio
1,aereo,1
2,rodoviario,2
3,fluvial,3
4,maritimo,4


# Tabelas centralizadoras

In [78]:
df_viagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322007 entries, 1 to 322007
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id_viagem                 322007 non-null  object        
 1   Situação                  322007 non-null  object        
 2   Viagem Urgente            322007 non-null  object        
 3   Código do órgão superior  322007 non-null  int64         
 4   Nome do órgão superior    322007 non-null  object        
 5   CPF viajante              322007 non-null  object        
 6   Nome                      322007 non-null  object        
 7   Cargo                     322007 non-null  object        
 8   Função                    322007 non-null  object        
 9   Descrição Função          322007 non-null  object        
 10  Período - Data de início  322007 non-null  datetime64[ns]
 11  Período - Data de fim     322007 non-null  datetime64[ns]
 12  Va

In [79]:
df_passagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107015 entries, 1 to 107015
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_viagem               107015 non-null  int64  
 1   Meio de transporte      107015 non-null  object 
 2   País - Origem ida       107015 non-null  object 
 3   UF - Origem ida         107015 non-null  object 
 4   Cidade - Origem ida     107015 non-null  object 
 5   País - Destino ida      107015 non-null  object 
 6   UF - Destino ida        107015 non-null  object 
 7   Cidade - Destino ida    107015 non-null  object 
 8   País - Origem volta     107015 non-null  object 
 9   UF - Origem volta       107015 non-null  object 
 10  Cidade - Origem volta   107015 non-null  object 
 11  Pais - Destino volta    107015 non-null  object 
 12  UF - Destino volta      107015 non-null  object 
 13  Cidade - Destino volta  107015 non-null  object 
 14  Valor da passagem   

## pagamento_table

In [80]:
df_pag_final = df_passagem.copy()
df_pag_final

,id_viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Total,id_passagem
1,16386658,aereo,brasil,rio grande do norte,natal,brasil,distrito federal,brasilia,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,522.04,1.14,523.18,1
2,16386658,aereo,brasil,distrito federal,brasilia,brasil,rio grande do norte,natal,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,529.57,1.14,530.71,2
3,16413295,aereo,brasil,sao paulo,sao paulo,brasil,distrito federal,brasilia,brasil,distrito federal,brasilia,brasil,sao paulo,sao paulo,627.42,1.14,628.56,3
4,16415470,aereo,brasil,rio de janeiro,rio de janeiro,portugal,sem informacao,lisboa,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,4
5,16415470,aereo,portugal,sem informacao,lisboa,brasil,rio de janeiro,rio de janeiro,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,3451.84,0.00,3451.84,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107011,2020000671,aereo,brasil,sao paulo,sao paulo,brasil,maranhao,sao luis,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,1084.34,0.00,1084.34,107011
107012,2020000672,aereo,brasil,distrito federal,brasilia,brasil,sao paulo,sao paulo,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,558.47,0.00,558.47,107012
107013,2020000704,aereo,brasil,parana,maringa,brasil,rio grande do sul,porto alegre,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,1791.07,0.00,1791.07,107013
107014,2020000704,aereo,brasil,rio grande do sul,porto alegre,brasil,parana,londrina,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,sem informacao,2357.85,0.00,2357.85,107014


In [81]:
df_pag_final = df_pag_final[['Valor da passagem', 'Taxa de serviço', 'id_passagem']]

In [84]:
df_pag_final['id_pagamento'] = df_pag_final.index

<ipython-input-84-f8d39a8e5866>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pag_final['id_pagamento'] = df_pag_final.index


In [85]:
df_pag_final

,Valor da passagem,Taxa de serviço,id_passagem,id_pagamento
1,522.04,1.14,1,1
2,529.57,1.14,2,2
3,627.42,1.14,3,3
4,3451.84,0.00,4,4
5,3451.84,0.00,5,5
...,...,...,...,...
107011,1084.34,0.00,107011,107011
107012,558.47,0.00,107012,107012
107013,1791.07,0.00,107013,107013
107014,2357.85,0.00,107014,107014


## PUC_table

### Substituição passagem table

In [86]:
df_passagem_final = df_passagem.copy()

In [87]:
# Subs pelo index de cada p,u,c
for i, row in df_passagem_final.iterrows():
    df_passagem_final.at[i, 'País - Origem ida'] = paises.index(row['País - Origem ida']) + 1
    df_passagem_final.at[i, 'País - Destino ida'] = paises.index(row['País - Destino ida']) + 1
    df_passagem_final.at[i, 'País - Origem volta'] = paises.index(row['País - Origem volta']) + 1
    df_passagem_final.at[i, 'Pais - Destino volta'] = paises.index(row['Pais - Destino volta']) + 1
    
    df_passagem_final.at[i, 'UF - Origem ida'] = uf.index(row['UF - Origem ida']) + 1
    df_passagem_final.at[i, 'UF - Destino ida'] = uf.index(row['UF - Destino ida']) + 1
    df_passagem_final.at[i, 'UF - Origem volta'] = uf.index(row['UF - Origem volta']) + 1
    df_passagem_final.at[i, 'UF - Destino volta'] = uf.index(row['UF - Destino volta']) + 1
    
    df_passagem_final.at[i, 'Cidade - Origem ida'] = cidades.index(row['Cidade - Origem ida']) + 1
    df_passagem_final.at[i, 'Cidade - Destino ida'] = cidades.index(row['Cidade - Destino ida']) + 1
    df_passagem_final.at[i, 'Cidade - Origem volta'] = cidades.index(row['Cidade - Origem volta']) + 1
    df_passagem_final.at[i, 'Cidade - Destino volta'] = cidades.index(row['Cidade - Destino volta']) + 1

In [88]:
df_passagem_final

,id_viagem,Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Total,id_passagem
1,16386658,aereo,20,13,537,20,6,534,5,5,387,5,5,387,522.04,1.14,523.18,1
2,16386658,aereo,20,6,534,20,13,537,5,5,387,5,5,387,529.57,1.14,530.71,2
3,16413295,aereo,20,84,528,20,6,534,20,6,534,20,84,528,627.42,1.14,628.56,3
4,16415470,aereo,20,17,38,121,5,82,5,5,387,5,5,387,3451.84,0.00,3451.84,4
5,16415470,aereo,121,5,82,20,17,38,5,5,387,5,5,387,3451.84,0.00,3451.84,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107011,2020000671,aereo,20,84,528,20,34,412,5,5,387,5,5,387,1084.34,0.00,1084.34,107011
107012,2020000672,aereo,20,6,534,20,84,528,5,5,387,5,5,387,558.47,0.00,558.47,107012
107013,2020000704,aereo,20,58,283,20,74,689,5,5,387,5,5,387,1791.07,0.00,1791.07,107013
107014,2020000704,aereo,20,74,689,20,58,580,5,5,387,5,5,387,2357.85,0.00,2357.85,107014


### puc table

In [89]:
# Criação lista puc
puc_lista = []

for i, row in df_passagem_final.iterrows():
    puc_origem_ida = (row['País - Origem ida'], row['UF - Origem ida'], row['Cidade - Origem ida'])
    puc_destino_ida = (row['País - Destino ida'], row['UF - Destino ida'], row['Cidade - Destino ida'])
    puc_origem_volta = (row['País - Origem volta'], row['UF - Origem volta'], row['Cidade - Origem volta'])
    puc_destino_volta = (row['Pais - Destino volta'], row['UF - Destino volta'], row['Cidade - Destino volta'])
    
    puc_lista.append(puc_origem_ida)
    puc_lista.append(puc_destino_ida)
    puc_lista.append(puc_origem_volta)
    puc_lista.append(puc_destino_volta)

In [90]:
df_puc_final = pd.DataFrame(puc_lista, columns=['id_pais', 'id_uf', 'id_cidade'])

In [91]:
df_puc_final = df_puc_final.drop_duplicates()
df_puc_final = df_puc_final.reset_index(drop=True)

In [92]:
df_puc_final.index +=1
df_puc_final['id_puc'] = df_puc_final.index

In [93]:
df_puc_final

,id_pais,id_uf,id_cidade,id_puc
1,20,13,537,1
2,20,6,534,2
3,5,5,387,3
4,20,84,528,4
5,20,17,38,5
...,...,...,...,...
691,20,51,548,691
692,20,57,324,692
693,75,5,274,693
694,101,68,330,694


## CPF_table

In [94]:
df_viagem_final = df_viagem.copy()

### Substituição viagem table

In [95]:
# Subs pelo index de cada c,p,f
pessoas = list(df_pessoa_final['CPF viajante'])
cargos = list(df_cargo['Cargo'])
funcoes = list(df_funcao['Função'])

for i, row in df_viagem_final.iterrows():
    df_viagem_final.at[i, 'CPF viajante'] = pessoas.index(row['CPF viajante']) + 1
    df_viagem_final.at[i, 'Cargo'] = cargos.index(row['Cargo']) + 1
    df_viagem_final.at[i, 'Função'] = funcoes.index(row['Função']) + 1

In [96]:
df_viagem_final

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Valor passagens
1,16332319,nao realizada,nao,26000,ministerio da educacao,1,claudia correa de andrade oliveira,1,1,sem informacao,2020-01-05,2021-01-04,0.00
2,16386658,realizada,nao,26000,ministerio da educacao,2,alexsandro galeno araujo dantas,1,2,fuc-0001,2020-03-10,2020-03-12,1053.89
3,16401038,realizada,nao,26000,ministerio da educacao,3,fernando carlos giacomelli,1,1,sem informacao,2020-01-01,2020-09-15,0.00
4,16413295,realizada,nao,26000,ministerio da educacao,4,rodrigo ramos hospodar felippe valverde,2,1,sem informacao,2020-03-02,2020-03-02,628.56
5,16415470,realizada,sim,26000,ministerio da educacao,5,martius vicente rodriguez y rodriguez,1,3,cargo comissionado de direcao,2020-01-11,2020-01-23,6903.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322003,2020000671,realizada,nao,25000,ministerio da economia,105762,antonio carlos pereira rosa,687,1,sem informacao,2020-11-30,2020-12-04,1934.81
322004,2020000672,realizada,nao,25000,ministerio da economia,105574,joao manoel pinho de mello,689,1,sem informacao,2020-12-10,2020-12-17,558.47
322005,2020000704,realizada,nao,25000,ministerio da economia,66331,juliano elizardo da silva,687,1,sem informacao,2020-12-14,2020-12-18,4148.92
322006,2020000712,realizada,nao,25000,ministerio da economia,105605,fabio kanczuk,689,1,sem informacao,2020-12-17,2020-12-18,1080.47


### cpf table

In [97]:
# Criação lista cpf
cpf_lista = []

for i, row in df_viagem_final.iterrows():
    cpf = (row['CPF viajante'], row['Cargo'], row['Função'])
    cpf_lista.append(cpf)

In [98]:
df_cpf = pd.DataFrame(cpf_lista, columns=['id_pessoa', 'id_cargo', 'id_funcao'])

In [99]:
df_cpf = df_cpf.drop_duplicates()
df_cpf = df_cpf.reset_index(drop=True)

In [100]:
df_cpf.index +=1
df_cpf['id_cpf'] = df_cpf.index

In [101]:
df_cpf

,id_pessoa,id_cargo,id_funcao,id_cpf
1,1,1,1,1
2,2,1,2,2
3,3,1,1,3
4,4,2,1,4
5,5,1,3,5
...,...,...,...,...
113455,105759,685,1,113455
113456,105760,687,1,113456
113457,105761,687,1,113457
113458,105762,687,1,113458


## passagem_table

In [102]:
df_passagem_final_2 = df_passagem_final.copy()

### Meio de transporte merge

In [103]:
df_passagem_final_2 = pd.merge(df_passagem_final_2, df_meio, left_on='Meio de transporte', how='left',right_on='nome')

In [104]:
df_passagem_final_2 = df_passagem_final_2.drop('Total', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Meio de transporte', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Valor da passagem', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('Taxa de serviço', axis=1)
df_passagem_final_2 = df_passagem_final_2.drop('nome', axis=1)

In [105]:
df_passagem_final_2.index += 1

In [106]:
df_passagem_final_2

,id_viagem,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,id_passagem,id_meio
1,16386658,20,13,537,20,6,534,5,5,387,5,5,387,1,1
2,16386658,20,6,534,20,13,537,5,5,387,5,5,387,2,1
3,16413295,20,84,528,20,6,534,20,6,534,20,84,528,3,1
4,16415470,20,17,38,121,5,82,5,5,387,5,5,387,4,1
5,16415470,121,5,82,20,17,38,5,5,387,5,5,387,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107011,2020000671,20,84,528,20,34,412,5,5,387,5,5,387,107011,1
107012,2020000672,20,6,534,20,84,528,5,5,387,5,5,387,107012,1
107013,2020000704,20,58,283,20,74,689,5,5,387,5,5,387,107013,1
107014,2020000704,20,74,689,20,58,580,5,5,387,5,5,387,107014,1


### PUC merge

In [107]:
df_puc_final

,id_pais,id_uf,id_cidade,id_puc
1,20,13,537,1
2,20,6,534,2
3,5,5,387,3
4,20,84,528,4
5,20,17,38,5
...,...,...,...,...
691,20,51,548,691
692,20,57,324,692
693,75,5,274,693
694,101,68,330,694


In [108]:
df_puc_merge = df_passagem_final_2.copy()

In [109]:
# Origem ida
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_origem_ida'})

In [110]:
# Destino ida
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_destino_ida'})

In [111]:
# Origem volta
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_origem_volta'})

In [112]:
# Destino volta
df_puc_merge = pd.merge(df_puc_merge, df_puc_final, 
                        left_on=['Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta'], 
                        right_on=['id_pais', 'id_uf', 'id_cidade'],
                        how='left')

df_puc_merge = df_puc_merge.rename(columns={'id_puc': 'id_puc_destino_volta'})

In [113]:
df_puc_merge = df_puc_merge[['id_passagem', 
                             'id_viagem', 
                             'id_meio', 
                             'id_puc_origem_ida', 
                             'id_puc_destino_ida', 
                             'id_puc_origem_volta', 
                             'id_puc_destino_volta']]

In [114]:
df_puc_merge.index += 1

In [115]:
df_passagem_final = df_puc_merge
df_passagem_final

,id_passagem,id_viagem,id_meio,id_puc_origem_ida,id_puc_destino_ida,id_puc_origem_volta,id_puc_destino_volta
1,1,16386658,1,1,2,3,3
2,2,16386658,1,2,1,3,3
3,3,16413295,1,4,2,2,4
4,4,16415470,1,5,6,3,3
5,5,16415470,1,6,5,3,3
...,...,...,...,...,...,...,...
107011,107011,2020000671,1,4,59,3,3
107012,107012,2020000672,1,2,4,3,3
107013,107013,2020000704,1,87,7,3,3
107014,107014,2020000704,1,7,20,3,3


## viagem_table

In [118]:
df_viagem_final_2 = df_viagem_final.copy()

### CPF merge

In [119]:
# Origem ida
df_cpf_merge = pd.merge(df_viagem_final_2, df_cpf, 
                        left_on=['CPF viajante', 'Cargo', 'Função'], 
                        right_on=['id_pessoa', 'id_cargo', 'id_funcao'],
                        how='left')

In [120]:
df_cpf_merge.index += 1

In [121]:
df_cpf_merge

,id_viagem,Situação,Viagem Urgente,Código do órgão superior,Nome do órgão superior,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Valor passagens,id_pessoa,id_cargo,id_funcao,id_cpf
1,16332319,nao realizada,nao,26000,ministerio da educacao,1,claudia correa de andrade oliveira,1,1,sem informacao,2020-01-05,2021-01-04,0.00,1,1,1,1
2,16386658,realizada,nao,26000,ministerio da educacao,2,alexsandro galeno araujo dantas,1,2,fuc-0001,2020-03-10,2020-03-12,1053.89,2,1,2,2
3,16401038,realizada,nao,26000,ministerio da educacao,3,fernando carlos giacomelli,1,1,sem informacao,2020-01-01,2020-09-15,0.00,3,1,1,3
4,16413295,realizada,nao,26000,ministerio da educacao,4,rodrigo ramos hospodar felippe valverde,2,1,sem informacao,2020-03-02,2020-03-02,628.56,4,2,1,4
5,16415470,realizada,sim,26000,ministerio da educacao,5,martius vicente rodriguez y rodriguez,1,3,cargo comissionado de direcao,2020-01-11,2020-01-23,6903.68,5,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322003,2020000671,realizada,nao,25000,ministerio da economia,105762,antonio carlos pereira rosa,687,1,sem informacao,2020-11-30,2020-12-04,1934.81,105762,687,1,113458
322004,2020000672,realizada,nao,25000,ministerio da economia,105574,joao manoel pinho de mello,689,1,sem informacao,2020-12-10,2020-12-17,558.47,105574,689,1,113248
322005,2020000704,realizada,nao,25000,ministerio da economia,66331,juliano elizardo da silva,687,1,sem informacao,2020-12-14,2020-12-18,4148.92,66331,687,1,113450
322006,2020000712,realizada,nao,25000,ministerio da economia,105605,fabio kanczuk,689,1,sem informacao,2020-12-17,2020-12-18,1080.47,105605,689,1,113282


### Finalização

In [122]:
df_viagem_final = df_cpf_merge[['id_viagem', 
                             'Situação', 
                             'Viagem Urgente', 
                             'Período - Data de início', 
                             'Período - Data de fim', 
                             'Código do órgão superior',
                             'id_cpf', 
                             'Valor passagens']]

In [123]:
df_viagem_final = df_viagem_final.rename(columns={"Situação": 'is_realizado', 
                                                  'Viagem Urgente': 'is_urgente', 
                                                  'Período - Data de início': 'data_inicio', 
                                                  'Período - Data de fim': 'data_fim',
                                                  'Código do órgão superior': 'id_ministerio', 
                                                  'Valor passagens': 'total'})

In [124]:
df_viagem_final['is_realizado'] = df_viagem_final['is_realizado'].map({"nao realizada": False, "realizada": True})
df_viagem_final['is_urgente'] = df_viagem_final['is_urgente'].map({"nao": False, "sim": True})

In [125]:
df_viagem_final

,id_viagem,is_realizado,is_urgente,data_inicio,data_fim,id_ministerio,id_cpf,total
1,16332319,False,False,2020-01-05,2021-01-04,26000,1,0.00
2,16386658,True,False,2020-03-10,2020-03-12,26000,2,1053.89
3,16401038,True,False,2020-01-01,2020-09-15,26000,3,0.00
4,16413295,True,False,2020-03-02,2020-03-02,26000,4,628.56
5,16415470,True,True,2020-01-11,2020-01-23,26000,5,6903.68
...,...,...,...,...,...,...,...,...
322003,2020000671,True,False,2020-11-30,2020-12-04,25000,113458,1934.81
322004,2020000672,True,False,2020-12-10,2020-12-17,25000,113248,558.47
322005,2020000704,True,False,2020-12-14,2020-12-18,25000,113450,4148.92
322006,2020000712,True,False,2020-12-17,2020-12-18,25000,113282,1080.47


# Resumo das tabelas

## Viagem, passagem e pagamento

In [126]:
df_viagem_final

,id_viagem,is_realizado,is_urgente,data_inicio,data_fim,id_ministerio,id_cpf,total
1,16332319,False,False,2020-01-05,2021-01-04,26000,1,0.00
2,16386658,True,False,2020-03-10,2020-03-12,26000,2,1053.89
3,16401038,True,False,2020-01-01,2020-09-15,26000,3,0.00
4,16413295,True,False,2020-03-02,2020-03-02,26000,4,628.56
5,16415470,True,True,2020-01-11,2020-01-23,26000,5,6903.68
...,...,...,...,...,...,...,...,...
322003,2020000671,True,False,2020-11-30,2020-12-04,25000,113458,1934.81
322004,2020000672,True,False,2020-12-10,2020-12-17,25000,113248,558.47
322005,2020000704,True,False,2020-12-14,2020-12-18,25000,113450,4148.92
322006,2020000712,True,False,2020-12-17,2020-12-18,25000,113282,1080.47


In [127]:
df_passagem_final

,id_passagem,id_viagem,id_meio,id_puc_origem_ida,id_puc_destino_ida,id_puc_origem_volta,id_puc_destino_volta
1,1,16386658,1,1,2,3,3
2,2,16386658,1,2,1,3,3
3,3,16413295,1,4,2,2,4
4,4,16415470,1,5,6,3,3
5,5,16415470,1,6,5,3,3
...,...,...,...,...,...,...,...
107011,107011,2020000671,1,4,59,3,3
107012,107012,2020000672,1,2,4,3,3
107013,107013,2020000704,1,87,7,3,3
107014,107014,2020000704,1,7,20,3,3


In [128]:
df_pag_final = df_pag_final.rename(columns={'Taxa de serviço': 'taxa', 'Valor da passagem': 'valor',})
df_pag_final = df_pag_final[['id_pagamento', 'valor', 'taxa', 'id_passagem']]

In [129]:
df_pag_final

,id_pagamento,valor,taxa,id_passagem
1,1,522.04,1.14,1
2,2,529.57,1.14,2
3,3,627.42,1.14,3
4,4,3451.84,0.00,4
5,5,3451.84,0.00,5
...,...,...,...,...
107011,107011,1084.34,0.00,107011
107012,107012,558.47,0.00,107012
107013,107013,1791.07,0.00,107013
107014,107014,2357.85,0.00,107014


## CPF e PUC

In [130]:
df_cpf = df_cpf[['id_cpf', 'id_pessoa', 'id_cargo', 'id_funcao']]

In [131]:
df_cpf

,id_cpf,id_pessoa,id_cargo,id_funcao
1,1,1,1,1
2,2,2,1,2
3,3,3,1,1
4,4,4,2,1
5,5,5,1,3
...,...,...,...,...
113455,113455,105759,685,1
113456,113456,105760,687,1
113457,113457,105761,687,1
113458,113458,105762,687,1


In [132]:
df_puc_final = df_puc_final[['id_puc', 'id_pais', 'id_uf', 'id_cidade',]]

In [133]:
df_puc_final

,id_puc,id_pais,id_uf,id_cidade
1,1,20,13,537
2,2,20,6,534
3,3,5,5,387
4,4,20,84,528
5,5,20,17,38
...,...,...,...,...
691,691,20,51,548
692,692,20,57,324
693,693,75,5,274
694,694,101,68,330


## Resto

In [134]:
df_paises = df_paises[['id_paises', 'nome']]

In [135]:
df_paises

,id_paises,nome
1,1,bulgaria
2,2,irlanda
3,3,costa rica
4,4,sri lanka
5,5,sem informacao
...,...,...
126,126,espanha
127,127,russia
128,128,canada
129,129,chipre


In [136]:
df_uf = df_uf[['id_uf', 'nome']]

In [137]:
df_uf

,id_uf,nome
1,1,bulgaria
2,2,paraiba
3,3,irlanda
4,4,pernambuco
5,5,sem informacao
...,...,...
87,87,portugal
88,88,mocambique
89,89,espanha
90,90,russia


In [138]:
df_cidades = df_cidades[['id_cidade', 'nome']]

In [139]:
df_cidades

,id_cidade,nome
1,1,hortolandia
2,2,tbilisi
3,3,xangai
4,4,aalborg
5,5,sofia
...,...,...
736,736,mendoza
737,737,chipre
738,738,russia
739,739,larnaca


In [140]:
df_pessoa_final = df_pessoa_final.rename(columns={'Nome': 'nome', 'CPF viajante': 'cpf',})
df_pessoa_final = df_pessoa_final[['id_pessoa', 'nome', 'cpf']]

In [141]:
df_pessoa_final

,id_pessoa,nome,cpf
1,1,claudia correa de andrade oliveira,***.211.458-**
2,2,alexsandro galeno araujo dantas,***.708.864-**
3,3,fernando carlos giacomelli,***.634.280-**
4,4,rodrigo ramos hospodar felippe valverde,***.464.507-**
5,5,martius vicente rodriguez y rodriguez,***.126.127-**
...,...,...,...
105758,105758,rodrigo campelo crispim,***.922.261-**
105759,105759,vinicius pacheco barcelos,***.679.700-**
105760,105760,cecilia silva gontijo,***.906.656-**
105761,105761,robson linko nunes da silva,***.873.083-**


In [142]:
df_cargo = df_cargo.rename(columns={'Cargo': 'nome', })
df_cargo = df_cargo[['id_cargo', 'nome']]

In [143]:
df_cargo

,id_cargo,nome
1,1,professor do magisterio superior
2,2,sem informacao
3,3,pesquisador em saude publica
4,4,professor ens basico tecn tecnologico
5,5,professor tit-livre magist superior
...,...,...
686,686,diretor servidor do banco central
687,687,tecnico do banco central
688,688,presidente do banco central
689,689,diretor do banco central


In [144]:
df_funcao = df_funcao.rename(columns={'Descrição Função': 'descricao', 'Função': 'nome'})
df_funcao = df_funcao[['id_funcao', 'nome', 'descricao']]

In [145]:
df_funcao

,id_funcao,nome,descricao
1,1,sem informacao,sem informacao
2,2,fuc-0001,fuc-0001
3,3,cd-0003,cargo comissionado de direcao
4,4,ofsuperior,ofsuperior
5,5,of.general,oficiais generais
...,...,...,...
138,138,gts-003,gratificacao temporaria - sipam
139,139,das-1033,sem informacao
140,140,das-1032,sem informacao
141,141,nes-0032,cargos de natureza especial


In [146]:
df_ministerios = df_ministerios.rename(columns={'nome_ministerio': 'nome'})

In [147]:
df_ministerios

,id_ministerio,nome
1,26000,ministerio da educacao
6,52000,ministerio da defesa
8,36000,ministerio da saude
30,35000,ministerio das relacoes exteriores
51,30000,ministerio da justica e seguranca publica
200,25000,ministerio da economia
283,37000,controladoria-geral da uniao
292,39000,ministerio da infraestrutura
330,32000,ministerio de minas e energia
369,-1,sem informacao


In [148]:
df_meio = df_meio[['id_meio', 'nome']]

In [149]:
df_meio.head()

,id_meio,nome
1,1,aereo
2,2,rodoviario
3,3,fluvial
4,4,maritimo


# Salvando

In [150]:
df_viagem_final.to_csv('./dataset/gerados/viagem.csv')
df_passagem_final.to_csv('./dataset/gerados/passagem.csv')
df_pag_final.to_csv('./dataset/gerados/pagamento.csv')
df_cpf.to_csv('./dataset/gerados/cpf.csv')
df_puc_final.to_csv('./dataset/gerados/puc.csv')
df_paises.to_csv('./dataset/gerados/paises.csv')
df_uf.to_csv('./dataset/gerados/uf.csv')
df_cidades.to_csv('./dataset/gerados/cidades.csv')
df_pessoa_final.to_csv('./dataset/gerados/pessoa.csv')
df_cargo.to_csv('./dataset/gerados/cargo.csv')
df_funcao.to_csv('./dataset/gerados/funcao.csv')
df_ministerios.to_csv('./dataset/gerados/ministerio.csv')
df_meio.to_csv('./dataset/gerados/meio.csv')